In [ ]:
PEData=PyCoTools.PEAnalysis.ParsePEData(K.PEData_pickle)

print 'best estimated parameters:\n',PEData.data.iloc[0].sort_index()
PyCoTools.pycopi.InsertParameters(K.kholodenko_model,ParameterPath=K.PEData_pickle,Index=0)
PE=PyCoTools.pycopi.ParameterEstimation(K.kholodenko_model,K.noisy_timecourse_report,
                                        Method='CurrentSolutionStatistics',
                                        Plot='true',
                                        SaveFig='false',
                                        RandomizeStartValues='false') #important to turn this off
PE.set_up() ## setup
PE.run()    ## and run the current solution statistics parameter estimation

Solutions include:
* Run more parameter estimations with different genetic algorithm specific settings (i.e. increase population size and number of generations for genetic algorithm)
* Choose a  different algorithm and start again
* Run a local optimization starting from the top $x$ best parameter sets.

The latter is my favorite since it has the advantage that information is not lost from the *primary* parameter estimations.The logic it that even if the global optimizers haven't foud the 'house in the neibourhood' of the minima (whether local or global), they probably have found the neighbourhood. Therefore 'chasing' global primary parameter estimations with secondary local estimations is a good way to push the estimates a little further. 

Before proceding with runing 'chaser' secondary parameter estimations, lets visualize the data we already have

In [ ]:
Scatter graphs are useful for identifying relationships between variables. Usually the most interesting are linear relationships which can (but not *always*) indicate structural non-identifiability and plots of the objective function versus estimated parameters. These tend to least to 'identifiability signatures' which can indicate the identifiability status of a parameter ahead of an identifiability analysis. 


As mentioned previously, secondary 'chaser' parameter estimations are good for pushing a global algorithm's results a little further and can sometimes be extremely informative. This process could be done manually for a select few parameter sets but realistically this isn't an option without programming tools. The general idea is the same as what would be done manually: insert parameters into a model then setup and run a parameter estimation using a local method. Here is a function which does just that. 

## Secondary 'Chaser' Parameter Estimations

In [ ]:
import os
import PyCoTools
import FilePaths

def runHJ(copasi_file,parameters,report_name,mode):
    '''
    copasi_file:
        file to run hook and jeeves on 
    parameters:
        dictionary of parameters. Keys must match model names, values are input parameters
    report_name:
        name of the report to output PE data.
    mode:
        either 'SGE' for SGE cluster or 'true' to run on current machine
    '''
    if 'RSS' in parameters.keys():
        del parameters['RSS']
    for i in parameters:
        print i,':\t',parameters[i]
    PyCoTools.pycopi.InsertParameters(copasi_file,ParameterDict=parameters)
    
    PE=PyCoTools.pycopi.ParameterEstimation(copasi_file,K.noisy_timecourse_report,
                                         Method='HookeJeeves',
                                         IterationLimit=20000,
                                         Tolerance=1e-30,
                                         RandomizeStartValues='false',
                                         Plot='true',
                                         SaveFig='true',
                                         UseTemplateStartValues='false',
                                         )

        
    PE.set_up()
    ## Run via scan task because this gives only best values in function
    ## evaluations, rather than the periodic function evaluations as well
    print '\n\n'
    print report_name
    print '\n'
    PyCoTools.pycopi.Scan(copasi_file,ScanType='repeat',Run=mode,
                          NumberOfSteps=1,
                          ReportName=report,
                          ReportType='parameter_estimation')
##Use the function
if __name__=='__main__':
    K=FilePaths.KholodenkoExample()
    data=PyCoTools.PEAnalysis.ParsePEData(K.PEData_pickle).data
    for i in range(int(data.shape[0]*0.025)):
        print 'running index {} with starting RSS of {}'.format(i,data.iloc[i]['RSS'])
        if os.path.isdir(K.local_PEData_dir)!=True:
            os.mkdir(K.local_PEData_dir)
        report=os.path.join(K.local_PEData_dir,'{}.txt'.format(i))
        print runHJ(K.kholodenko_model,data.iloc[i].to_dict(),report,'true')